In [ ]:
from flask import Flask, render_template, Response, redirect, url_for, request
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime
from openpyxl import Workbook, load_workbook

app = Flask(__name__)

# تحميل جميع الصور من مجلد 'photo/' والتعرف على الوجوه
face_encodings_list = []
face_names = []

photo_dir = "photo/"
if not os.path.exists(photo_dir):
    print(f"⚠️ خطأ: المجلد '{photo_dir}' غير موجود.")
    exit()

for file_name in os.listdir(photo_dir):
    if file_name.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(photo_dir, file_name)
        image = face_recognition.load_image_file(image_path)
        encodings = face_recognition.face_encodings(image)
        if encodings:
            face_encodings_list.append(encodings[0])
            face_names.append(os.path.splitext(file_name)[0])
        else:
            print(f"⚠️ تحذير: لم يتم العثور على وجه في {file_name}")

if not face_encodings_list:
    print("⚠️ خطأ: لم يتم تحميل أي وجوه، تأكد من وضع صور صالحة في المجلد.")
    exit()

# قائمة لجميع الطلاب (نفترض أن الصور تمثل الطلاب)
all_students = face_names.copy()
# قائمة لتخزين الطلاب الذين تم التعرف عليهم في جلسة واحدة (اليوم)
attended_students = []
# قاموس لتتبع آخر يوم تم تسجيل حضور كل طالب لتجنب التكرار في نفس اليوم
last_attendance_day = {student: None for student in all_students}
# قاموس لتتبع الحضور الشهري لكل طالب
monthly_attendance = {student: 0 for student in all_students}
# قاموس لتتبع حالة الدفع لكل طالب (False: لم يدفع، True: دفع)
payment_status = {student: False for student in all_students}
# قاموس لتخزين طريقة الدفع لكل طالب (مثلاً "Vodafone Cash", "Insta Pay", "Cash")
payment_method = {student: None for student in all_students}

# متغير لتحديد حالة التسجيل (تشغيل/إيقاف)
capturing = False

# قيمة الرسوم لكل طالب (يمكن تعديلها من خلال صفحة تعديل الرسوم)
fee = 100

# بدء التقاط الفيديو من الكاميرا
video_cap = cv2.VideoCapture(0)
video_cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
video_cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

def gen_frames():
    """توليد إطارات الفيديو لمعالجتها وبثها عبر الويب."""
    global attended_students, capturing, last_attendance_day
    while True:
        ret, frame = video_cap.read()
        if not ret:
            break

        if capturing:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb)
            face_encodings = face_recognition.face_encodings(rgb, face_locations)
            face_names_detected = []
            today = datetime.now().strftime("%Y-%m-%d")

            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(face_encodings_list, face_encoding, tolerance=0.5)
                face_distances = face_recognition.face_distance(face_encodings_list, face_encoding)
                if any(matches):
                    best_match = np.argmin(face_distances)
                    name = face_names[best_match]
                else:
                    name = "Unknown"
                face_names_detected.append(name)
                
                # تسجيل الحضور إذا لم يكن "Unknown" ولم يتم تسجيله اليوم مسبقاً
                if name != "Unknown":
                    if last_attendance_day.get(name) != today:
                        attended_students.append(name)
                        last_attendance_day[name] = today

            # رسم مستطيل حول الوجوه مع كتابة الاسم
            for (top, right, bottom, left), name in zip(face_locations, face_names_detected):
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, name, (left, top - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Attendance Not Started", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        ret, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

@app.route('/')
def index():
    """عرض الصفحة الرئيسية مع بث الفيديو وأزرار التحكم."""
    return render_template('index.html', capturing=capturing, fee=fee)

@app.route('/video_feed')
def video_feed():
    """بث الفيديو المُعالج."""
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/start')
def start_attendance():
    """بدء تسجيل الحضور."""
    global capturing
    capturing = True
    return redirect(url_for('index'))

@app.route('/end')
def end_attendance():
    """إنهاء تسجيل الحضور وتحديث ملف Excel مع بيانات الحضور، حالة الدفع، وطريقة الدفع، وعمود 'اجمالي'."""
    global capturing, attended_students, monthly_attendance
    capturing = False
    today = datetime.now().strftime("%Y-%m-%d")
    
    # تحديث الحضور الشهري: لكل طالب مسجل اليوم، زيادة واحدة فقط
    for student in set(attended_students):
        monthly_attendance[student] += 1

    excel_file = "Attendance.xlsx"
    if os.path.exists(excel_file):
        wb = load_workbook(excel_file)
        ws = wb.active
    else:
        wb = Workbook()
        ws = wb.active
        ws.cell(row=1, column=1, value="Student")
        for i, student in enumerate(all_students, start=2):
            ws.cell(row=i, column=1, value=student)
    
    # تحديث عمود الحضور لهذا اليوم (تاريخ اليوم)
    col_attendance = None
    for c in range(2, ws.max_column + 1):
        if ws.cell(row=1, column=c).value == today:
            col_attendance = c
            break
    if not col_attendance:
        col_attendance = ws.max_column + 1
        ws.cell(row=1, column=col_attendance, value=today)
    
    for i, student in enumerate(all_students, start=2):
        ws.cell(row=i, column=col_attendance, value=True if student in set(attended_students) else False)
    
    # تحديث عمود "Payment Status"
    payment_col = None
    for c in range(2, ws.max_column + 1):
        if ws.cell(row=1, column=c).value == "Payment Status":
            payment_col = c
            break
    if not payment_col:
        payment_col = ws.max_column + 1
        ws.cell(row=1, column=payment_col, value="Payment Status")
    
    for i, student in enumerate(all_students, start=2):
        status = "Paid" if payment_status.get(student, False) else "Not Paid"
        ws.cell(row=i, column=payment_col, value=status)
    
    # تحديث عمود "Payment Method"
    method_col = None
    for c in range(2, ws.max_column + 1):
        if ws.cell(row=1, column=c).value == "Payment Method":
            method_col = c
            break
    if not method_col:
        method_col = ws.max_column + 1
        ws.cell(row=1, column=method_col, value="Payment Method")
    
    for i, student in enumerate(all_students, start=2):
        method = payment_method.get(student, "") if payment_status.get(student, False) else ""
        ws.cell(row=i, column=method_col, value=method)
    
    # تحديث عمود "اجمالي"
    total_col = None
    for c in range(2, ws.max_column + 1):
        if ws.cell(row=1, column=c).value == "اجمالي":
            total_col = c
            break
    if not total_col:
        total_col = ws.max_column + 1
        ws.cell(row=1, column=total_col, value="اجمالي")
    
    for i, student in enumerate(all_students, start=2):
        total_value = fee if payment_status.get(student, False) else 0
        ws.cell(row=i, column=total_col, value=total_value)
    
    wb.save(excel_file)
    attended_students = []  # إعادة تعيين قائمة الحضور اليومية
    return redirect(url_for('index'))

@app.route('/mark_paid/<student>', methods=['GET', 'POST'])
def mark_paid(student):
    """صفحة لتحديد طريقة الدفع للطالب وتحديث حالة الدفع."""
    global payment_status, payment_method
    if request.method == 'POST':
        # قراءة طريقة الدفع المحددة من النموذج
        method = request.form.get('payment_method')
        if student in payment_status:
            payment_status[student] = True
            payment_method[student] = method
        return redirect(url_for('attendance_status'))
    return render_template('mark_paid.html', student=student)

@app.route('/attendance_status')
def attendance_status():
    """عرض حالة الحضور الشهري وحالة الدفع وطريقة الدفع لكل طالب."""
    monthly_data = []
    for student in all_students:
        count = monthly_attendance.get(student, 0)
        message = ""
        if count < 6:
            message = f"لم يكمل الشهر ({count}/6)"
        pay_status = "Paid" if payment_status.get(student, False) else "Not Paid"
        method = payment_method.get(student, "") if payment_status.get(student, False) else ""
        monthly_data.append({"name": student, "count": count, "message": message, "paid": pay_status, "method": method})
    return render_template('attendance.html', monthly_data=monthly_data, fee=fee)

@app.route('/total_payment')
def total_payment():
    """حساب وإظهار إجمالي المدفوع بناءً على عدد الطلاب الذين دفعوا."""
    global fee
    paid_count = sum(1 for s in payment_status if payment_status[s])
    total = paid_count * fee
    return render_template('total_payment.html', paid_count=paid_count, fee=fee, total=total)

@app.route('/set_fee', methods=['GET', 'POST'])
def set_fee():
    """تعديل قيمة الرسوم لكل طالب."""
    global fee
    if request.method == 'POST':
        try:
            fee = float(request.form.get('fee'))
        except (ValueError, TypeError):
            fee = 100
        return redirect(url_for('index'))
    return render_template('set_fee.html', fee=fee)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.6:5000/ (Press CTRL+C to quit)
192.168.1.6 - - [19/Feb/2025 11:45:52] "GET / HTTP/1.1" 200 -
192.168.1.6 - - [19/Feb/2025 11:45:53] "GET /video_feed HTTP/1.1" 200 -
192.168.1.6 - - [19/Feb/2025 11:46:03] "GET /start HTTP/1.1" 302 -
192.168.1.6 - - [19/Feb/2025 11:46:03] "GET / HTTP/1.1" 200 -
192.168.1.6 - - [19/Feb/2025 11:46:06] "GET /video_feed HTTP/1.1" 200 -
192.168.1.6 - - [19/Feb/2025 11:46:11] "GET /end HTTP/1.1" 302 -
192.168.1.6 - - [19/Feb/2025 11:46:11] "GET / HTTP/1.1" 200 -
192.168.1.6 - - [19/Feb/2025 11:46:11] "GET /video_feed HTTP/1.1" 200 -
192.168.1.6 - - [19/Feb/2025 11:46:13] "GET /attendance_status HTTP/1.1" 200 -
192.168.1.6 - - [19/Feb/2025 11:46:15] "GET /mark_paid/eman HTTP/1.1" 200 -
192.168.1.6 - - [19/Feb/2025 11:46:22] "POST /mark_paid/eman HTTP/1.1" 302 -
192.168.1.6 - - [19/Feb/2025 11:46:22] "GET /attendance_status HTTP/1.1" 200 -
192.168.1.6 - - [19/Feb/2025 11:46:26] "GET /mark_paid/mahoud HT